In [5]:
# caricamento dati da K-NN
from helper_rasar_simple_alpha import *

X_try, _, _, y_train, y_test, _ = load_data_rasar('lc_db_processed.csv', 'binary')

# _, X_train, _, y_train, _, _ = load_data_rasar('lc_db_processed.csv', 'binary')

# del _


In [6]:
X_try

,obs_duration_mean,conc1_type,exposure_type,control_type,media_type,application_freq_unit,class,tax_order,family,genus,...,pub872,pub873,pub874,pub875,pub876,pub877,pub878,pub879,pub880,pub881
6896,96.0,2,7,9,1,8,1,18,18,101,...,0,0,0,0,0,0,0,0,0,0
19797,96.0,2,1,9,1,8,1,9,28,60,...,0,0,0,0,0,0,0,0,0,0
14432,48.0,1,7,5,1,8,1,8,25,7,...,0,0,0,0,0,0,0,0,0,0
2357,24.0,1,7,5,1,8,1,10,66,160,...,0,0,0,0,0,0,0,0,0,0
4664,48.0,1,7,5,1,8,1,18,61,66,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8918,72.0,2,7,3,1,8,1,9,28,152,...,0,0,0,0,0,0,0,0,0,0
12110,24.0,2,7,9,1,8,1,9,28,152,...,0,0,0,0,0,0,0,0,0,0
19984,24.0,1,1,9,2,8,1,10,29,68,...,0,0,0,0,0,0,0,0,0,0
16470,96.0,1,7,4,1,8,1,21,69,134,...,0,0,0,0,0,0,0,0,0,0


# SIMPLE RASAR

In [2]:
cv_simple_rasar_alpha(X_try, np.concatenate([y_train, y_test]))

Computing distance matrix Sat Mar 20 21:54:17 2021
Epoch 1:  Sat Mar 20 22:00:59 2021
Epoch 2:  Sat Mar 20 22:03:10 2021
Epoch 3:  Sat Mar 20 22:06:37 2021
Epoch 4:  Sat Mar 20 22:09:41 2021
Epoch 5:  Sat Mar 20 22:12:25 2021
...END Simple RASAR Sat Mar 20 22:15:08 2021
Accuracy:    0.9131061146029072 se: 0.0020231509943683516
Sensitivity: 0.9214146615863793 se: 0.003168908416700975
Specificity: 0.8997743564555775 se: 0.0034253900707390864
F1 score:    0.9289905022933752 se: 0.0017247646520609935


### ottimizzo i parametri della regressione logistica interna

In [2]:
hyper_params_tune = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['elasticnet'],
    'max_iter': list(range(100,800,100)),
    'l1_ratio': [int(i)/10 for i in range(0,11,1)],
    'solver': ['saga'],
    'fit_intercept': [True, False]
}

tab = cv_params_simple_rasar_alpha(X_train, y_train, hyper_params_tune)

Tue Mar 23 19:10:13 2021
Epoch 1:  Tue Mar 23 19:15:38 2021
Epoch 2:  Tue Mar 23 19:16:41 2021
Epoch 3:  Tue Mar 23 19:17:38 2021
Epoch 4:  Tue Mar 23 19:18:36 2021
Epoch 5:  Tue Mar 23 19:19:34 2021
Tue Mar 23 19:20:32 2021


,solver,penalty,max_iter,l1_ratio,fit_intercept,C,test_acc,test_sens,test_spec,test_f1
mod0,saga,elasticnet,600,0.5,False,0.001,0.384131,0.000000,1.000000,0.000000
mod1,saga,elasticnet,400,0.5,False,1000.000,0.901743,0.939799,0.840606,0.921768
mod2,saga,elasticnet,600,0.1,False,0.001,0.623137,0.997593,0.022770,0.765276
mod3,saga,elasticnet,700,0.2,False,1000.000,0.901743,0.939799,0.840606,0.921768
mod4,saga,elasticnet,400,0.6,True,0.100,0.887579,0.956816,0.776826,0.912900
...,...,...,...,...,...,...,...,...,...,...
mod145,saga,elasticnet,700,0.8,False,100.000,0.901743,0.939799,0.840606,0.921768
mod146,saga,elasticnet,200,0.5,True,100.000,0.903152,0.913068,0.887198,0.920692
mod147,saga,elasticnet,400,1.0,False,0.001,0.384131,0.000000,1.000000,0.000000
mod148,saga,elasticnet,200,0.6,True,0.010,0.615869,1.000000,0.000000,0.762264


In [12]:
tab[tab.test_acc == tab.test_acc.max()].to_dict('records')[0]
params_tuned = tab[tab.test_acc == tab.test_acc.max()].to_dict('records')[0]
del params_tuned['test_acc']
del params_tuned['test_sens']
del params_tuned['test_spec']
del params_tuned['test_f1']
print(params_tuned)

{'solver': 'saga', 'penalty': 'elasticnet', 'max_iter': 100, 'l1_ratio': 0.8, 'fit_intercept': True, 'C': 1.0}


In [2]:
# interpretazioni al contrario di specificity e recall
params_tuned = {'solver': 'saga', 'penalty': 'elasticnet', 'max_iter': 100,
                'l1_ratio': 0.8, 'fit_intercept': True, 'C': 1.0}
print(params_tuned)
cv_simple_rasar_alpha(X_try, np.concatenate([y_train, y_test]), hyper_params = params_tuned)

{'solver': 'saga', 'penalty': 'elasticnet', 'max_iter': 100, 'l1_ratio': 0.8, 'fit_intercept': True, 'C': 1.0}
Computing distance matrix Tue Mar 23 19:38:55 2021
Epoch 1:  Tue Mar 23 19:45:52 2021
Epoch 2:  Tue Mar 23 19:48:27 2021
Epoch 3:  Tue Mar 23 19:52:38 2021
Epoch 4:  Tue Mar 23 19:56:12 2021
Epoch 5:  Tue Mar 23 19:58:31 2021
...END Simple RASAR Tue Mar 23 20:01:39 2021
Accuracy:    0.9127086854699116 se: 0.0019390904766548058
Sensitivity: 0.9182710043329753 se: 0.0029207913081512075
Specificity: 0.9037861040240266 se: 0.0025628461971642148
F1 score:    0.9284621207086456 se: 0.001667491828267933


# DF RASAR

In [1]:
from helper_rasar_datafusion_alpha import *

db_mortality, db_datafusion = load_data_rasar_datafusion('lc_db_processed.csv', 'datafusion_db_processed.csv')
print(ctime())
db_label = create_label_rasar('lc_db_processed.csv', 'datafusion_db_processed.csv')
cv_datafusion_rasar_alpha(db_mortality, db_datafusion, db_label)

Sun Apr  4 13:02:30 2021
Computing distance matrix Sun Apr  4 13:16:57 2021
Epoch 1:  Sun Apr  4 13:22:13 2021
Epoch 2:  Sun Apr  4 13:27:21 2021
Epoch 3:  Sun Apr  4 13:33:03 2021
Epoch 4:  Sun Apr  4 13:38:53 2021
Epoch 5:  Sun Apr  4 13:44:47 2021
Accuracy:    0.9227942473302416 se: 0.0012582361221227137
Sensitivity: 0.8847362774326436 se: 0.0031939991471836944
Specificity: 0.9464491219785325 se: 0.001370554939467216
F1 score:    0.8977247909579715 se: 0.001543393632641932


### ottimizzo i parametri della random forest interna

In [1]:
from helper_rasar_datafusion_alpha import *

db_mortality, db_datafusion = load_data_rasar_datafusion('lc_db_processed.csv', 'datafusion_db_processed.csv')

X_train, _, y_train, _ = train_test_split(db_mortality.drop(columns = ['conc1_mean']), 
                                                    db_mortality['conc1_mean'],
                                                    test_size=0.33, random_state=42) 
del _

X_train.loc[:, 'conc1_mean'] = y_train

print(ctime())
db_label = create_label_rasar('lc_db_processed.csv', 'datafusion_db_processed.csv')

hyper_parameters_tune = {'n_estimators': [int(i) for i in np.linspace(100, 1000, 10)],
                         'max_depth': [int(i) for i in np.linspace(10, 30, 10)],
                         'min_samples_split': [2, 5, 10], 
                         'min_samples_leaf': [1, 2, 4],
                         'bootstrap': [True, False],
                         'random_state': [3456]}

tab = cv_params_datafusion_rasar_alpha(X_train, db_datafusion, db_label, hyper_parameters_tune)

Sun Apr  4 17:51:54 2021
Sun Apr  4 18:06:38 2021
Computing distance matrix Sun Apr  4 18:06:38 2021
Epoch 1:  Sun Apr  4 18:09:35 2021
Epoch 2:  Sun Apr  4 18:32:57 2021
Epoch 3:  Sun Apr  4 18:55:54 2021
Epoch 4:  Sun Apr  4 19:18:49 2021
Epoch 5:  Sun Apr  4 19:41:59 2021
Sun Apr  4 20:04:58 2021


In [2]:
tab[tab.test_acc == tab.test_acc.max()].to_dict('records')[0]
params_tuned = tab[tab.test_acc == tab.test_acc.max()].to_dict('records')[0]
del params_tuned['test_acc']
del params_tuned['test_sens']
del params_tuned['test_spec']
del params_tuned['test_f1']
print(params_tuned)

{'random_state': 3456, 'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 30, 'bootstrap': False}


In [7]:
del X_train, hyper_parameters_tune, y_train, tab
cv_datafusion_rasar_alpha(db_mortality, db_datafusion, db_label,
                          params = {'random_state': 3456, 'n_estimators': 300, 'min_samples_split': 10,
                                    'min_samples_leaf': 2, 'max_depth': 30, 'bootstrap': False})

Computing distance matrix Sun Apr  4 20:33:48 2021
Epoch 1:  Sun Apr  4 20:39:10 2021
Epoch 2:  Sun Apr  4 20:44:24 2021
Epoch 3:  Sun Apr  4 20:49:37 2021
Epoch 4:  Sun Apr  4 20:54:51 2021
Epoch 5:  Sun Apr  4 21:00:10 2021
Accuracy:    0.9234897760827909 se: 0.0008863770414557245
Sensitivity: 0.8863733302273452 se: 0.002589610753786875
Specificity: 0.9465253898885368 se: 0.001297572568448857
F1 score:    0.8987072283670037 se: 0.001369428851698919
